# Instance creator utility

data  
|-->info  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    |-->name  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    |-->description  
|-->network  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    |-->nodes  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        |-->node 0  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        .  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        .  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    |-->info  
|-->requests  
|-->fleet  

In [12]:
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
def TSubElement(parent, tag, attrib={}, text=None, **extra):
    element = ET.SubElement(parent, tag, attrib, **extra)
    if text:
        element.text = text
    return element

### XML tree related functions

In [3]:
def createInfo(root, name='', description=''):
    info = TSubElement(root, 'info')
    _name = TSubElement(info, 'name', text=name)
    _description = TSubElement(info, 'description', text=description)
    return

def createNetwork(root, numDepots, numCustomers, numChargeStations, 
                  cx_low=-1.0, cx_high=1.0, cy_low=-1.0, cy_high=1.0, 
                  request_min=0.01, request_max=0.8, 
                  tw_low_low=600.0, tw_low_high=960.0, tw_min_width= 20.0, tw_max_width=120.0, 
                  travel_time_min=3., travel_time_max=35., 
                  energy_consumption_min=2.0, energy_consumption_max=10.0,
                  max_capacity_charge_stations=4):
    
    network = TSubElement(root, 'network')
    nodes = TSubElement(network, 'nodes') # stores the nodes
    arcs = TSubElement(network, 'arcs') # stores arcs
    info = TSubElement(network, 'info') # stores info about the network
    technologies = TSubElement(network, 'technologies') # stores info about CS techonologies
    
    # depot nodes
    for i in range(numDepots):
        attr = {'id': str(i), 'type': str(0)}
        node = TSubElement(nodes, 'node', attrib=attr)

    # customer nodes
    for i in range(i+1, i+numCustomers+1):
        attr = {'id': str(i), 'type': str(1)}
        node = TSubElement(nodes, 'node', attrib=attr)
        
        request = np.random.uniform(request_min, request_max)
        _request = TSubElement(node, 'request', text='{:.2f}'.format(request))
        
        tw_low = np.random.uniform(tw_low_low, tw_low_high)
        tw_upp = tw_low + np.random.uniform(tw_min_width, tw_max_width)
        _tw_low = TSubElement(node, 'tw_low', text='{:.2f}'.format(tw_low))
        _tw_upp = TSubElement(node, 'tw_upp', text='{:.2f}'.format(tw_upp))

    # CS nodes
    for i in range(i+1, i+numChargeStations+1):
        attr = {'id': str(i), 'type': str(2)}
        node = TSubElement(nodes, 'node', attrib=attr)
        
        technology = np.random.randint(1, 4) # choose among 1, 2 and 3
        _technology = TSubElement(node, 'technology', text=str(technology))
        
        capacity = np.random.randint(1, max_capacity_charge_stations+1)
        _capatity = TSubElement(node, 'capacity', text=str(capacity))
    
    # add coordinates (random)
    for node in nodes:
        cx = np.random.uniform(cx_low, cx_high)
        cy = np.random.uniform(cy_low, cy_high)
        _cx = TSubElement(node, 'cx', text='{:.2f}'.format(cx))
        _cy = TSubElement(node, 'cy', text='{:.2f}'.format(cy))
        
    # Arcs
    for i in nodes:
        nodeFrom = TSubElement(arcs, 'node_from', attrib={'id': i.get('id')})
        for j in nodes:
            nodeTo = TSubElement(nodeFrom, 'node_to', attrib={'id': j.get('id')})
            
            if i == j:
                travelTime = 0.
                energyConsumption = 0.
            else:
                travelTime = np.random.uniform(travel_time_min, travel_time_max)
                energyConsumption = np.random.uniform(energy_consumption_min, energy_consumption_max)
                
            _travelTime = TSubElement(nodeTo, 'travel_time', text='{:.2f}'.format(travelTime))
            _energyConsumption = TSubElement(nodeTo, 'energy_consumption', text='{:.2f}'.format(energyConsumption))
            
    # technologies
    _tech1 = TSubElement(technologies, 'technology', attrib={'type': str(1)})
    _tech2 = TSubElement(technologies, 'technology', attrib={'type': str(2)})
    _tech3 = TSubElement(technologies, 'technology', attrib={'type': str(3)})
    
    tech1 = {'0.0': '0.0', '20.0': '75.0', '50.0': '80.0', '120.0': '100.0'}
    tech2 = {'0.0': '0.0', '15.0': '75.0', '40.0': '80.0', '80.0': '100.0'}
    tech3 = {'0.0': '0.0', '10.0': '75.0', '25.0': '80.0', '40.0': '100.0'}
    
    _tech_list = [_tech1, _tech2, _tech3]
    tech_list = [tech1, tech2, tech3]
    
    for _tech, tech in zip(_tech_list, tech_list):
        for chargingTime, battLevel in tech.items():
            breakPoint = TSubElement(_tech, 'breakpoint')
            _chargingTime = TSubElement(breakPoint, 'charging_time', text=chargingTime)
            _battLevel = TSubElement(breakPoint, 'battery_level', text= battLevel)
        
def createRequests(root):
    nodes = root.find('network').find('nodes')
    for node in nodes:
        if node.get('type') == '1':
            pass
    return

def createFleet(root, numEV, attrib):
    _fleet = TSubElement(root, 'fleet')
    _numEV = TSubElement(_fleet, 'fleet_size', text=str(numEV))
    _attributes = TSubElement(_fleet, 'vehicle_attributes')
    for attr, val in attrib.items():
        _attr = TSubElement(_attributes, str(attr), text=str(val))
    return

def createTree(numDepots, numCustomers, numChargeStations, numEV, attribsEV):
    dataElement = ET.Element('instance')
    
    infoElement = createInfo(dataElement)
    networkElement = createNetwork(dataElement, numDepots, numCustomers, numChargeStations)
    requestsElement = createRequests(dataElement)
    
    fleetElement = createFleet(dataElement, numEV, attribsEV)
    
    return ET.ElementTree(dataElement)

In [4]:
numDepot = 1
numCustomer = 5
numCS = 4

numEV = 4
attrib = {'max_payload': 1.5, 'max_tour_duration': 250.0}

dataTree = createTree(numDepot, numCustomer, numCS, numEV, attrib)

instanceName = 'd'+str(numDepot)+'c'+str(numCustomer)+'cs'+str(numCS)+'_ev'+str(numEV)
dataTree.write(instanceName+'.xml')

In [5]:
# usage example
nodes = dataTree.find('network').find('nodes')
arcs = dataTree.find('network').find('arcs')

f = 0
t = 7

print('travel time from', f, 'to', t, ':', float(arcs[f][t][0].text)) # travel time [f] to [t]
print('energy consumption from', f, 'to', t, ':', float(arcs[f][t][1].text)) # energy consumption [f] to [t]

travel time from 0 to 7 : 26.35
energy consumption from 0 to 7 : 2.53


In [11]:
# example of matrices construction
networkSize = len(nodes)

timeMatrix = np.zeros((networkSize, networkSize))
energyMatrix = np.zeros((networkSize, networkSize))
coordinates = [0]*networkSize

for i, nodeFrom in enumerate(arcs):
    for j, nodeTo in enumerate(nodeFrom):
        timeMatrix[i][j] = float(nodeTo[0].text)
        energyMatrix[i][j] = float(nodeTo[1].text)
    coordinates[i] = (float(nodes[i].find('cx').text), float(nodes[i].find('cy').text))
        
print('RESULTING TIME MATRIX:\n', timeMatrix)
print('RESULTING ENERGY CONSUMPTION MATRIX:\n', energyMatrix)
print('RESULTING NODES COORDINATES:\n',coordinates)

RESULTING TIME MATRIX:
 [[ 0.   27.58 22.62 34.   32.89 29.01 15.65 26.35 20.03  4.82]
 [26.21  0.   16.86 21.05 19.16 22.49 18.22 12.29 26.03  9.78]
 [ 5.27 20.22  0.   22.55 21.35 23.42 16.71 14.24 34.96 28.09]
 [30.69 17.86 31.1   0.    8.53 12.57 16.27 18.33 10.05 14.26]
 [21.97 24.44 28.61 13.57  0.   10.57 26.23 28.64 33.44 23.07]
 [26.11 18.14  6.69 32.01  4.96  0.   13.01 15.93  4.66 19.87]
 [30.68 27.94  7.45 31.95 24.19 13.24  0.    5.57 30.67  5.64]
 [11.47 32.97 16.22  6.97 13.24 19.22 20.17  0.   24.65  9.13]
 [28.98 26.77 12.97 21.34  7.3  16.84 32.21 30.28  0.   25.51]
 [25.63 10.57 10.12 25.32  8.94 23.42 17.86 32.63 30.97  0.  ]]
RESULTING ENERGY CONSUMPTION MATRIX:
 [[0.   7.27 8.7  6.03 9.07 2.67 5.89 2.53 8.22 8.58]
 [2.96 0.   2.81 8.77 5.14 4.47 2.78 5.43 4.5  8.49]
 [6.5  5.96 0.   4.8  2.47 4.19 3.27 9.37 7.94 2.1 ]
 [8.76 9.96 9.02 0.   5.06 7.93 4.95 5.6  6.27 5.93]
 [8.81 7.21 5.94 2.89 0.   3.61 2.22 7.25 4.59 7.95]
 [5.91 2.36 6.59 7.35 7.85 0.   2.34 7.32 

In [13]:
# show a graph of resulting network
G = nx.Graph()
